In [1]:
import pandas as pd
import numpy as np
from os import path

In [2]:
data=pd.read_table('Last_FM/lastfm_small.tsv', names=['user', 'artist','artist_name', 'total_plays'] )

In [3]:
data.fillna(0, inplace=True)

In [4]:
data['user_id']=data.user.astype('category').cat.codes.copy()+1
data['artist_id']=data.artist.astype('category').cat.codes.copy()+1

In [5]:
data.drop(['user','artist','artist_name'], axis=1,inplace=True)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
data_Test, data_Train=train_test_split(data, test_size=0.2, random_state=0 )

In [8]:
test_file_name = "lastfm.test.0"
data_Test[["user_id", "artist_id", "total_plays"]].to_csv(
    path.join(test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
data_Train[["user_id", "artist_id", "total_plays"]].to_csv(
    path.join(train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [9]:
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [10]:
from scipy.sparse import coo_matrix
plays_knn=coo_matrix((data_Train['total_plays'].astype(np.float32),(data_Train['artist_id'], data_Train['user_id'])))
sparse_info(plays_knn.tocsr())

Размерности матрицы: (46553, 8876)
Ненулевых элементов в матрице: 86784
Доля ненулевых элементов: 0.00021002679090360572
Среднее значение ненулевых элементов: 217.2936553955078
Максимальное значение ненулевых элементов: 39655.0
Минимальное значение ненулевых элементов: 1.0


In [11]:
plays_als=coo_matrix((data_Train['total_plays'].astype(np.double),(data_Train['artist_id'], data_Train['user_id'])))
sparse_info(plays_als.tocsr())

Размерности матрицы: (46553, 8876)
Ненулевых элементов в матрице: 86784
Доля ненулевых элементов: 0.00021002679090360572
Среднее значение ненулевых элементов: 217.29366012168143
Максимальное значение ненулевых элементов: 39655.0
Минимальное значение ненулевых элементов: 1.0


In [12]:
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

In [13]:
model_knn=CosineRecommender()
model_knn.fit(plays_knn)
sparse_info(model_knn.similarity)

Размерности матрицы: (46553, 46553)
Ненулевых элементов в матрице: 297594
Доля ненулевых элементов: 0.00013731842836818496
Среднее значение ненулевых элементов: 0.3843267302606184
Максимальное значение ненулевых элементов: 1.0000000593353844
Минимальное значение ненулевых элементов: 0.0


In [84]:
model_als=AlternatingLeastSquares(factors=20,regularization=0.01, iterations=300)
model_als.fit(plays_als)

In [40]:
user_plays = plays_knn.T.tocsr()
with open(("recs_knn.tsv"), "w") as output_file:
    for user_id in data_Test["user_id"].unique():
        for artist_id, score in model_knn.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

In [85]:
user_plays = plays_als.T.tocsr()
with open(("lastfm.test.0.recs.tsv"), "w") as output_file:
    for user_id in data_Test["user_id"].unique():
        for artist_id, score in model_als.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))